In [ ]:
import osgeo
import os
from os.path import join as pjoin
import rioxarray
import xarray as xr
import numpy as np
#import matplotlib as plt
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from rasterio import features
import pandas as pd
import rasterio

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns


scratch_dir = './data' 

In [ ]:
# open the 2009 classifications folder
aoi = gpd.read_file("data/Train/classifications/Classifications.shp")
aoi = aoi[['layer', 'is_glacier', 'geometry']]

# change the 0 to a 2 because later in the process all 0's will be removed
aoi['is_glacier'].loc[aoi['is_glacier']==0] = 2
print (aoi.tail())

In [ ]:
# get the training data
file_location = 'data/train_data_resampledNDSI.tif'
d = rioxarray.open_rasterio(file_location).squeeze() 
d.coords['band'] = d.coords['band'] +2008
d  # Show data information

In [ ]:
# We also prepare the color codes for visualization
colors = [(257, 71, 27), (98, 93, 78)]
cols = []
for col in colors:
    pal = sns.light_palette(col, input="husl", n_colors=4)
    for rgb in pal[1:]:
        cols.append(rgb)
# Assign color codes to LULC types 
symbology = {'Glacier': cols[2],
             'Non Glacier': cols[0]}

In [ ]:
# Labels from the AOIs
aoi['aoi_cat'] = pd.Categorical(aoi['is_glacier'])

# Rasterize
rst = rasterio.open('data/train_data_resampledNDSI.tif')  # Base image to rasterize the *.shp
meta = rst.meta.copy()  # Copy metadata from the base image
meta.update(compress='lzw')

# Burn the AOIs *.shp file into raster and save it
out_rst = 'data/train/aoi_rasterizedNDSI.tif'
with rasterio.open(out_rst, 'w+', **meta) as out:
    out_arr = out.read(1)

    # Create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(aoi.geometry, aoi.aoi_cat))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)

stack = np.array([])

print(d.shape)  # Each time, also check the size of the image
    
# In order to do clustering, image should be reshaped into a single column
band_col = d.values.reshape(-1, 1)
   
# Each time put the reshaped image into the stack
stack = np.hstack((stack, band_col)) if stack.size else band_col
# Also to check the size of the stack
print(stack.shape)

In [ ]:
# get the validation classes
aoi2 = gpd.read_file("data/Val/classifications/valclass1999.shp")
aoi2 = aoi2[['layer', 'is_glacier', 'geometry']]

# change the 0 to a 2 because later in the process all 0's will be removed
aoi2['is_glacier'].loc[aoi2['is_glacier']==0] = 2
print (aoi2.tail())

# get the validation data
file_location2 = 'data/validation_data_resampledNDSI.tif'
d2 = rioxarray.open_rasterio(file_location2).squeeze() 
d2.coords['band'] = d2.coords['band'] +1998

# Labels from the AOIs
aoi2['aoi_cat'] = pd.Categorical(aoi2['is_glacier'])

# Rasterize
rst2 = rasterio.open('data/validation_data_resampledNDSI.tif')  # Base image to rasterize the *.shp
meta = rst2.meta.copy()  # Copy metadata from the base image
meta.update(compress='lzw')

# Burn the AOIs *.shp file into raster and save it
out_rst = 'data/val/aoi_rasterizedNDSI.tif'
with rasterio.open(out_rst, 'w+', **meta) as out:
    out_arr = out.read(1)

    # Create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(aoi.geometry, aoi.aoi_cat))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)

stack2 = np.array([])

print(d2.shape)  # Each time, also check the size of the image
    
# In order to do clustering, image should be reshaped into a single column
band_col2 = d2.values.reshape(-1, 1)
   
# Each time put the reshaped image into the stack
stack2 = np.hstack((stack2, band_col2)) if stack2.size else band_col2
# Also to check the size of the stack
print(stack2.shape)

In [ ]:
# Load the rasterized LULC types in the AOI and concatenate it together with the images

aoi_rst = rasterio.open('data/train/aoi_rasterizedNDSI.tif').read(1)
aoi_rst2 = rasterio.open('data/val/aoi_rasterizedNDSI.tif').read(1)

# Stack the label with the input bands
data = np.c_[stack, aoi_rst.reshape(-1,)]
data2 = np.c_[stack2, aoi_rst2.reshape(-1,)]

# Of course, we are only interested in pixels with LULC type labelled
data = data[np.where(data[:,data.shape[1]-1]!=0)]
data2 = data2[np.where(data2[:,data2.shape[1]-1]!=0)]


X_train = data[:, :1] 
Y_train = data[:, 1:].reshape(-1,) 

X_test = data2[:, :1] 
Y_test = data2[:, 1:].reshape(-1,)  


In [ ]:
# rename the classes
aoi['is_glacier'].loc[aoi['is_glacier'] == 1] = 'Glacier'
aoi['is_glacier'].loc[aoi['is_glacier'] == 2] = 'Non Glacier'
aoi2['is_glacier'].loc[aoi2['is_glacier'] == 1] = 'Glacier'
aoi2['is_glacier'].loc[aoi2['is_glacier'] == 2] = 'Non Glacier'

In [ ]:
# Create a decision tree object
dt = DecisionTreeClassifier()

# Define the hyperparameter grid
param_grid = {'max_depth': [2, 3, 4, 5, 6, 8, 10],
              'min_samples_split': [2,3, 4,5, 6, 8, 10],
              'min_samples_leaf': [1, 2, 3, 4, 5]}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(dt, param_grid, cv=5)
grid_search.fit(X_train, Y_train)

# Print the best hyperparameters
print("Best hyperparameters: ", grid_search.best_params_)


In [ ]:
# Initiate a tree model
tree_depth = grid_search.best_params_['max_depth']
min_sample_split= grid_search.best_params_['min_samples_split'] 
min_samples_leaf= grid_search.best_params_['min_samples_leaf'] 
model_tree = DecisionTreeClassifier(max_depth=tree_depth, min_samples_split=min_sample_split, min_samples_leaf=min_samples_leaf)

# Fit the model to your data.
# Please note that the output of this fitting is a model with several parameters that are configurable, so far you only configured "max_depth" while training/fitting.
model_tree.fit(X_train, Y_train)

# Visualize split
fig = plt.figure(figsize=(tree_depth*4,tree_depth*3))
tree.plot_tree(model_tree, filled=True)

In [ ]:
Y_pred = pd.Series(list(model_tree.predict(X_test[:,0].reshape(-1,1))), name='DT prediction')  # Store the predicted value in Y_pred
Y_actu = pd.Series(list(Y_test), name='Manual delineation')

# Map the LULC codes to the actual name of LULC types

# First we need a mapping from the LULC codes to the actual LULC type name.
code_lulc = { 1: 'Glacier',
              2: 'Non Glacier'}

# Now replace the non-intuitive numbers with actual LULC type names and store them into new variables
Y_actu2 = Y_actu.replace(code_lulc)
Y_pred2 = Y_pred.replace(code_lulc)

# Show the LULC coded confusion matrix
df_confusion2 = pd.crosstab(Y_actu2, Y_pred2)
df_confusion2

In [ ]:
# Print out more detailed accuracy assessment report

from sklearn.metrics import classification_report

print(classification_report(Y_actu2, Y_pred2))

In [ ]:
Y_pred_all = model_tree.predict(stack2)

In [ ]:
# Visualize
fig1,(ax1, ax2) = plt.subplots(1,2, figsize=(20,10))
d2.plot(ax=ax1, cmap='gray', alpha=0.25, add_colorbar=False)  # Plot the satellite image
aoi2.plot(ax=ax1, column='land_cover', legend=True, color=aoi2['is_glacier'].map(symbology))

from matplotlib.lines import Line2D
custom_points = [Line2D([0], [0], marker="o", linestyle="none", markersize=5, color=color) for color in symbology.values()]
leg_points = ax1.legend(custom_points, symbology.keys(), loc='upper right', frameon=False)
ax1.add_artist(leg_points)

# Assign color codes to LULC types 
symbology2 = {1: cols[2],
              2: cols[0]}

from matplotlib.colors import ListedColormap
cm = ListedColormap(symbology2.values())
imin = min(symbology2)  # Colormap range
imax = max(symbology2)

# Visualize
# Because the predicted labels are still in one column, you need to reshape it back to original image shape
row, col = d2.shape  # Get the original dimensions of the image
imin = min(symbology2)  # Colormap range
imax = max(symbology2)

print('Printing large image takes time...')
ax2.imshow(Y_pred_all.reshape(row, col), cmap=cm, interpolation='none', vmin=imin, vmax=imax)

In [ ]:
# load the deployment data
file_location3 = 'data/resampledNDSI.tif'
data_all = rioxarray.open_rasterio(file_location3).squeeze() 
data_all.coords['band'] = data_all.coords['band'] +1991
print(data_all.shape)

# remove the artificialy created na's during the resampling
data_all = data_all.dropna(dim = 'band', how = 'all')


stack_all = np.array([])

print(data_all.shape)  # Each time, also check the size of the image
    
# In order to do clustering, image should be reshaped into a single column
band_col_all = data_all.values.reshape(-1, 1)
   
# Each time put the reshaped image into the stack
stack_all = np.hstack((stack_all, band_col_all)) if stack_all.size else band_col_all
# Also to check the size of the stack
print(stack_all.shape)

In [ ]:
# check is there is missing data, impute this with iterative imputer
np.isnan(stack_all).sum()
imp = IterativeImputer(max_iter = 10, random_state=42)
stack_all = imp.fit_transform(stack_all)

In [ ]:
# deploy model
Y_pred_all = model_tree.predict(stack_all)

In [ ]:
# save the results
results = xr.DataArray(Y_pred_all.reshape(data_all.shape),
                       coords ={'time':data_all.coords['band'], 
                                'y': data_all.coords['y'], 
                                'x': data_all.coords['x']}, 
                                dims=['band', 'y', 'x'])
results.rio.to_raster(pjoin(scratch_dir, 'resultsNDSI.tif'), compress='LZW')